In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd

## Open Geojson

In [2]:
germany_gdf = gpd.read_file(
    "data/georef-germany-kreis@public/georef-germany-kreis-millesime.shp"
)

In [3]:
germany_gdf["krs_code"] = germany_gdf["krs_code"].str[2:7]

## Get Open Population data

In [29]:
population_2016_df = pd.read_csv(
    "data/12521-0040_en_flat-2016/12521-0040_en_flat.csv",
    delimiter=";",
    dtype={"1_variable_attribute_code": str},
)
population_2016_df = population_2016_df[
    population_2016_df["2_variable_attribute_label"] == "Total"
].reset_index(drop=True)

In [30]:
population_2016_df = population_2016_df[population_2016_df["time"] == "2016-12-31"][
    ["1_variable_attribute_code", "value"]
].reset_index(drop=True)

In [40]:
population_2024_df = pd.read_csv(
    "data/12521-0040_en_flat-2024/12521-0040_en_flat.csv",
    delimiter=";",
    dtype={"1_variable_attribute_code": str},
)
population_2024_df = population_2024_df[
    population_2024_df["2_variable_attribute_label"] == "Total"
].reset_index(drop=True)

In [41]:
len(population_2016_df)

476

In [42]:
population_2024_df = population_2024_df[population_2024_df["time"] == "2024-12-31"][
    ["1_variable_attribute_code", "value"]
].reset_index(drop=True)

In [43]:
pop_df = population_2016_df.merge(
    population_2024_df,
    on="1_variable_attribute_code",
    how="inner",
    suffixes=("_2016", "_2024"),
)

In [44]:
len(pop_df)

476

In [45]:
pop_df = pop_df.rename(
    columns={
        "value_2016": "2016",
        "value_2024": "2024",
        "1_variable_attribute_code": "krs_code",
    }
)

In [46]:
pop_df["2016"] = pd.to_numeric(pop_df["2016"], errors="coerce")
pop_df["2024"] = pd.to_numeric(pop_df["2024"], errors="coerce")

In [47]:
pop_df["DIFF"] = pop_df["2024"] - pop_df["2016"]
pop_df["PERCENT"] = ((pop_df["DIFF"] / pop_df["2016"]) * 100.0).round(decimals=1)

## Merge data and export

In [48]:
pop_gdf = germany_gdf.merge(pop_df, on="krs_code", how="left")

In [49]:
pop_gdf = pop_gdf.to_crs(4839)
pop_gdf.to_file("data/Population_Change_Foreigners_In_Germany_2016-2024.gpkg")

In [50]:
pop_gdf["2016"].sum()

np.float64(10036140.0)

In [51]:
pop_gdf["2024"].sum()

np.float64(14061630.0)

In [52]:
pop_gdf["lan_code"] = pop_gdf["lan_code"].str[2:4]

In [53]:
pop_gdf.dissolve(by="lan_code")[["geometry"]].to_file("data/german_states.gpkg")

In [54]:
pop_gdf[pop_gdf["krs_code"] == "10046"]

,year,lan_code,lan_name,krs_code,krs_name,krs_area_co,krs_type,krs_name_sh,geometry,2016,2024,DIFF,PERCENT
138,2023,10,['Saarland'],10046,['Landkreis St. Wendel'],DEU,Landkreis,['St. Wendel'],"POLYGON ((-233057 -156205, -233133.055 -156259...",NaN,NaN,NaN,NaN
